#### This file creates correctness.csv, which stores the correctness marks of each student

In [ ]:
import os
import math
import numpy as np
import pandas as pd 

In [ ]:
home_dir: str = os.getcwd()
logs_dir: str = os.path.join(home_dir, 'logs')
plot_dir: str = os.path.join(home_dir, 'plots')
sub_dir: str = os.path.join(home_dir, 'subA3')
info_task1_dir: str = os.path.join(home_dir, 'info_task1')
info_task2_dir: str = os.path.join(home_dir, 'info_task2') 
results_dir: str = os.path.join(home_dir, 'results')

print (home_dir, logs_dir, plot_dir, sub_dir, results_dir)

Get kerberos from the entry number

In [ ]:
def get_kerberos(Entry_no: str) -> str:
    year, dept, endwith = Entry_no[2: 4], Entry_no[4: 7], Entry_no[7: ]
    kerberos = dept + year + endwith
    return kerberos.lower()

Decide the weights of each test case

In [ ]:
weights = {'small': 60, 'medium': 40} # Every test of equal ? 
task1_categories = {'small': [0, 4, 5], 'medium': [10]}
task2_categories = {'small': [0, 1, 5], 'medium': [10, 11, 12]}
total_test_cases = {'small': 0, 'medium': 0}

categories = ['small', 'medium'] # TODO: Change this accordingly the set of categories active.

for category in categories: # 'large']:
    if category in task1_categories:
        total_test_cases[category] = len(task1_categories[category]) + len(task2_categories[category])


Get the mapping of pairs from studentPairs.csv

In [ ]:
all_students = os.listdir(os.path.join(home_dir, 'subA3'))
all_students = [student.split('_')[1] for student in all_students]

For task1
1. Fetch the startk and endk for each test case.
2. For each test case within category hold the equal weight

In [ ]:
# 1. Fetch the startk and endk for each test case 
range_of_k = dict()
marksFor = dict()
for testcases in task1_categories.values():
    for testid in testcases:
        info_file = (os.path.join(info_task1_dir, f'test{testid}', 'task1_info.txt'))
        if os.path.exists(info_file):
            startk, endk = None, None
            with open(info_file, 'r') as f:
                line: str = f.readline().split(',')
                startk: int = int((line[0]).split('=')[1])
                endk: int = int((line[1]).split('=')[1])
                f.close()
            range_of_k[testid] = (startk, endk)
  
# 2. For each test case within category hold the equal weight
for category, testcases in task1_categories.items():
    uniform_weight = weights[category] / total_test_cases[category]
    for testid in testcases:
        marksFor[testid] = uniform_weight

# 3. Get the columns for the dataframes
alltestcases = []
for testcases in task1_categories.values():
    for testid in testcases:
        alltestcases.append(f'task1_test{testid}')
for testcases in task2_categories.values():
    for testid in testcases:
        alltestcases.append(f'task2_test{testid}')

# 4. Make the dataframe
student_df = pd.DataFrame(columns= ['Entry_no'] + alltestcases + ['Total Score'])
student_df['Entry_no'] = all_students
student_df.set_index('Entry_no', inplace=True)
student_df = student_df.replace(np.nan, 0)

# 5. For each k within the testcase, this hold for task1 only
final_marks = dict()
for testid, (startk, endk) in range_of_k.items():
    if not(testid in marksFor):
        continue
    marks = marksFor[testid]
    total_subpart = endk - startk + 1
    subpart_marks = marks / total_subpart
    final_marks[testid] = dict()
    for k in range(startk, endk + 1):
        final_marks[testid][k] = subpart_marks

print (marksFor)
print (final_marks)   
# 6. Filter the results of each student
student_marks = {testlabel:dict() for testlabel in alltestcases}

for testcases in task1_categories.values():
    for testid in testcases:
        test_logs_dir = os.path.join(logs_dir, f'task1_logs_{testid}')
        if os.path.exists(test_logs_dir):
            verdict_file = os.path.join(test_logs_dir, f'time_test_{testid}.csv')
            df = pd.read_csv(verdict_file)
            startk, endk = range_of_k[testid]
            columns = [f'Verdict GroupSize={k}' for k in range(startk, endk + 1)]
            for _, student in df.iterrows():
                entry_no = student['Entry_number']
                _entry_no = entry_no.replace('.txt', '')
                sum = 0
                for verdict in columns:
                    _k = int(verdict.split('=')[1])
                    if student[verdict] == 'AC':
                        sum += float(final_marks[testid][_k])
                    if student[verdict] == 'PC':
                        sum += float(final_marks[testid][_k] / 2)
                student_marks[f'task1_test{testid}'][_entry_no] = round(sum, 2)

student_marks

For task2 

In [ ]:
# 1. Fetch the k and p for each test case 
k, p = -1, -1
marksFor = dict() 
meta_data = dict()
for testcases in task2_categories.values():
    for testid in testcases:
        info_file = (os.path.join(info_task2_dir, f'test{testid}', 'task2_info.txt'))
        if os.path.exists(info_file):
            k, p = None, None
            with open(info_file, 'r') as f:
                line: str = f.readline().split(',')
                k: int = int((line[0]).split('=')[1])
                p: int = int((line[1]).split('=')[1])
                f.close()
            meta_data[testid] = (k, p)
            
# 2. For each test case within category hold the equal weight
for category, testcases in task2_categories.items():
    uniform_weight = weights[category] / total_test_cases[category]
    for testid in testcases:
        marksFor[testid] = uniform_weight

print(marksFor)
# 3. There are single k, p only. Filter the results of each student
for testcases in task2_categories.values():
    for testid in testcases:
        test_logs_dir = os.path.join(logs_dir, f'task2_logs_{testid}')
        if os.path.exists(test_logs_dir):
            verdict_file = os.path.join(test_logs_dir, f'time_test_{testid}.csv')
            df = pd.read_csv(verdict_file)
            k, p = meta_data[testid]
            # Since, there is just a single column in results
            verdict = f'Verdict: GroupSize={k}, and p = {p}'
            for _, student in df.iterrows():
                entry_no = student['Entry_number']
                _entry_no = entry_no.replace('.txt', '')
                score = 0
                if student[verdict] == 'AC':
                    score = float(marksFor[testid])
                elif student[verdict] == 'PC':
                    score = float(marksFor[testid] / 2)
                student_marks[f'task2_test{testid}'][_entry_no] = round(score, 2)

student_marks

Finally save the file to the result folder

In [ ]:
for testlabel, scores in student_marks.items():
    for entry_no, score in scores.items():
        student_df.loc[entry_no, testlabel] = score 

cols = list(student_df.columns)
cols.remove('Total Score')
student_df['Total Score'] = student_df.loc[:, cols].sum(axis='columns')
student_df['kerberos'] = student_df.index.map(get_kerberos)
student_df.reset_index(inplace=True)
student_df.set_index('kerberos', inplace=True)
student_df.to_csv(os.path.join(results_dir, 'correctness.csv'))
student_df

In [ ]:
stats_df = pd.read_csv(os.path.join(results_dir, 'correctness.csv'))
marks = stats_df['Total Score'] * (3.6 / 100)
# print (marks)
distribution = marks.describe()
print(distribution)